In [71]:
import requests
from pprint import pprint
from fake_useragent import UserAgent

session = requests.session()

response = session.get('https://www.lanazione.it/cronaca?page=1')

ua = UserAgent(verify_ssl=False)

headers = {'User-Agent': ua.random}
#print(headers)
response = session.get('https://www.lanazione.it/cronaca?page=1', headers=headers)

{'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36'}


Обкачаем немного новостей с сайта.

1. Страницы имеют вид "https://www.lanazione.it/cronaca?page={page_number}", поэтому можно просто идти циклом.
2. Достанем заголовок, краткое описание (из станицы со списком новостей), текст полной статьи (из самой страницы новости)
3. Положим в базу

In [72]:
import sqlite3
from bs4 import BeautifulSoup
import re

In [73]:
conn = sqlite3.connect('la_nazione_news.db')
cur = conn.cursor()

Создаем базу данных, где будем хранить информацию

In [74]:
conn = sqlite3.connect('la_nazione_news.db')
cur = conn.cursor()
cur.execute("""
CREATE TABLE IF NOT EXISTS texts 
(id INTEGER PRIMARY KEY AUTOINCREMENT, news_id text, title text, short_text text, full_text text)
""")

conn.commit()
conn.close()

**Шаг 1. Найти страницы**

Посмотрим, как устроены новости и скачаем одну страницу

In [159]:
page_number = 1
url = f'https://www.lanazione.it/cronaca?page={page_number}'
req = session.get(url, headers={'User-Agent': ua.random})
page = req.text
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
#print(soup.prettify())

#отдельные посты
news = soup.find_all('div', {'class': 'card__text'}) # news = soup.find_all('header', {'class': 'card__title'})
#len(news)

#Заголовок-ссылка и текст заголовка
title_obj = news[0].find('a', {'class': 'title__link'})
#title_obj
title = title_obj.text
#title

#достаем ссылку
attrs = title_obj.attrs
#attrs
href = title_obj.attrs['href']
#href

#короткий текст новости
short_text =  news[0].find('div', {'class': 'abstract'}).text
#short_text

#парсим страничку одной новости
url_one = href
#url_one

req = session.get(url_one, headers={'User-Agent': ua.random})
page_n = req.text
page_n = requests.get(url_one)

soup = BeautifulSoup(page_n.text, 'html.parser')
#print(soup.prettify())

#сохраняем текст статьи
full_text = soup.find('article', {'class': 'sc-1ig42x7-6 iSFXhF'}).text
#full_text[:500]


"Arezzo, 30 marzo 2022 -\xa0Dimenticare i rimpianti del derby per sfruttare la seconda chance casalinga e agganciare i maremmani al terzo posto. Ha più di un motivo l’Arezzo per battere il Tiferno Lerchi nel turno infrasettimanale. \nMariotti inserisce Van Der Velden al posto dello squalificato Biondi. A centrocampo tornano dall'inizio Marchi e Pizzutelli, completa il reparto Mancino. Attacco inedito con Doratiotto dietro Cutolo e Calderini.\xa0L'allenatore degli ospiti Machi schiera\xa0i suoi con il colla"

**Здесь я попыталась лемматизировать только текст одной новости** (но что-то пошло не так)

In [ ]:
articles_texts = full_text
#articles_texts

In [ ]:
articles_preprocessed = []
for a_text in articles_texts:
    a_tokens = wordpunct_tokenize(a_text)
    a_lemmatized = " ".join([morph.parse(item)[0].normal_form for item in a_tokens])
    articles_preprocessed.append(a_lemmatized)
print (articles_preprocessed)

**Здесь я оформила ранее ивлеченные данные в формулы**

In [125]:
#функция для парсинга страницы со списокм новостей (блок одной новости)
def parse_news_page_block(one_block):
    block = {}
    a = one_block.find('a', {'class': 'title__link'})
    block['title'] = a.text
    block['href'] = a.attrs['href']
    block['short_text'] = one_block.find('div', {'class': 'abstract'}).text
    return block

page_number = 1
url = f'https://www.lanazione.it/cronaca?page={page_number}'
req = session.get(url, headers={'User-Agent': ua.random})
page = req.text
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
news = soup.find_all('div', {'class': 'card__text'})
# for n in news: 
#     print(parse_news_page_block(n))


In [138]:
#функция для парсинга отдельной страницы новости
def parse_one_article(block):
    url_one = block['href']
    req = session.get(url_one, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    block['full_text'] = soup.find('article', {'class': 'sc-1ig42x7-6 iSFXhF'}).text
    return block

page_number = 1
url = f'https://www.lanazione.it/cronaca?page={page_number}'
req = session.get(url, headers={'User-Agent': ua.random})
page = req.text
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
news = soup.find_all('div', {'class': 'card__text'})
print(parse_one_article(parse_news_page_block(news[1])))


{'title': "Trovato morto tra la differenziata: l'autopsia, decesso per compressione", 'href': 'https://www.lanazione.it/empoli/cronaca/morto-empoli-carta-1.7519532', 'short_text': "\n                    Empoli, la vicenda dell'extracomunitario rinvenuto cadavere in un'azienda che tratta la carta: il giovane si sarebbe addormentato nel contenitore\n            ", 'full_text': "Empoli, 30 marzo 2022 - Si avvalora l'ipotesi che la morte del giovane somalo trovato in un contenitore della differenziata sia morto per una tragica fatalità. Ovvero che si sia addormentato in un cassonetto salvo poi essere inconsapevolmente portato via con il camion dagli addetti. L'autopsia infatti rivela, nelle sue prime risultanze, che il ragazzo è morto a causa di una compressione.\xa0Il cadavere è stato rinvenuto da un operaio in una ditta cartaria di Empoli che stava proprio smistanto il materiale cartaceo arrivato dalla raccolta differenziata. Operando con la gru ha visto il corpo. Comprensibile lo choc d

**Здесь я попыталась лемматизировать постранично, но безуспешно**

In [152]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [142]:
from nltk.tokenize import wordpunct_tokenize
from pymorphy2 import MorphAnalyzer

In [143]:
morph = MorphAnalyzer()

In [144]:
#функция для парсинга отдельной страницы новости
def parse_one_article(block):
    url_one = block['href']
    req = session.get(url_one, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    block['full_text'] = soup.find('article', {'class': 'sc-1ig42x7-6 iSFXhF'}).text
    return block

page_number = 1
url = f'https://www.lanazione.it/cronaca?page={page_number}'
req = session.get(url, headers={'User-Agent': ua.random})
page = req.text
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
news = soup.find_all('div', {'class': 'card__text'})
#print(parse_one_article(parse_news_page_block(news[1])))

In [156]:
articles_texts = parse_one_article(parse_news_page_block(news[1]))

In [157]:
articles_preprocessed = []
for a_text in articles_texts:
    a_tokens = wordpunct_tokenize(a_text)
    a_lemmatized = " ".join([morph.parse(item)[0].normal_form for item in a_tokens])
    articles_preprocessed.append(a_lemmatized)
print (articles_preprocessed)

['title', 'href', 'short_text', 'full_text']


In [150]:
articles_tfidf = tfidf.fit_transform(articles_preprocessed)
print(f"Матрица на {articles_tfidf.shape[0]} документов и {articles_tfidf.shape[1]} термов")

NameError: name 'tfidf' is not defined

Регулярное выражение для того, чтобы достать ID новости и не повторяться

In [127]:
regex_news_id = re.compile('/([0-9]*?).html')

Обработать N-ую страницу новостей

In [137]:
seen_news = []
exceptions = []
def get_nth_page(page_number):
    # скачиваем
    url = f'https://www.lanazione.it/cronaca?page={page_number}'
    req = session.get(url, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    
    # находим новости
    news = soup.find_all('div', {'class': 'card__text'})
    
    # идем по новостям и обрабатываем их
    blocks = []
    for n in news:
        try:
            blocks.append(parse_news_page_block(n))
        except Exception as e:
            print(e)

    # идем по отдельным статьям и достаем информацию
    result = []
    for b in blocks:
        if b['href'].startswith('/'):
            idx = regex_news_id.findall(b['href'])[0]
            if idx not in seen_news:
                try:
                    res = parse_one_article(b)
                    res['news_id'] = idx
                    result.append(res)
                except Exception as e:
                    print(e)
            else:
                print('Seen', b['href'])
    
    # возвращаем найденную информацию
    return result

# print(get_nth_page(1))

[]


**Здесь я попыталась записать в базу данных данные, которые я ранее оформила в функции**

In [129]:
conn = sqlite3.connect('la_nazione_news1.db')
cur = conn.cursor()
cur.execute("""
CREATE TABLE IF NOT EXISTS texts 
(id INTEGER PRIMARY KEY AUTOINCREMENT, news_id text, title text, short_text text, full_text text)
""")

conn.commit()
conn.close()

In [130]:
def write_to_db(block):

    # сохраняем информацию по текстам
    cur.execute(
        """
        INSERT INTO texts 
            (news_id, title, short_text, full_text) 
            VALUES (?, ?, ?, ?)
        """, (
            block['news_id'],
            block['title'], block['short_text'], block['full_text'])
    )
    
    # достаем id текста
    cur.execute("SELECT id FROM texts WHERE news_id = ?", (block['news_id'],))
    text_id = cur.fetchone()[0]
    
    conn.commit()
    
    # добавляем, что такой id уже видели
    seen_news.add(block['news_id'])

In [131]:
conn = sqlite3.connect('la_nazione_news1.db')
cur = conn.cursor()

In [132]:
# достаем, какие новости уже видели (если потом захотим обновлять базу)
cur.execute('SELECT news_id FROM texts')
seen_news = set(i[0] for i in cur.fetchall())

In [133]:
from tqdm.auto import tqdm

Напишем функцию, куда передаем количество страниц и она выполняет все нужные действия

In [134]:
def run_all(n_pages):
    for i in tqdm(range(n_pages)):
        blocks = get_nth_page(i+1)
        for block in blocks:
            write_to_db(block)

Запускаем на 20 первых страниц

In [139]:
run_all(1)

100%|██████████| 1/1 [00:01<00:00,  1.40s/it]


## tf-idf

In [ ]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
stops = stopwords.words("italian")

tfidf = TfidfVectorizer(
    analyzer="word", # анализировать по словам или по символам (char)
    stop_words=stops # передаём список стоп-слов для русского из NLTK
)

In [ ]:
from nltk.tokenize import wordpunct_tokenize
from pymorphy2 import MorphAnalyzer

In [ ]:
morph = MorphAnalyzer()

In [ ]:
conn = sqlite3.connect('la_nazione_news1.db')
cur = conn.cursor()
cur.execute("""
SELECT full_text FROM texts
""")
all_texts = cur.fetchall()
#print(all_texts)

In [ ]:
texts_preprocessed = []
punct = '.,?!»«№/#%*()—-["]@'
for text in all_texts:
    for a_text in text:
        a_tokens = wordpunct_tokenize(a_text)
        for item in a_tokens:
            if item not in punct:
                a_lemmatized = " ".join([morph.parse(item)[0].normal_form])
                texts_preprocessed.append(a_lemmatized)
#print(texts_preprocessed)